In [1]:
from mymodule.views.starcoordinatesview import StarCoordinatesView

In [2]:
file_path = "mymodule/resources/cereal_data_set/cereal.csv"
view = StarCoordinatesView(file_path)
view.run()

Loading BokehJS ...

mymodule/resources/cereal_data_set/cereal.csv
